In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder  ## Dummify
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.compose import ColumnTransformer  ## Related to pipeline



from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier


from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt 

import seaborn as sns


from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score

In [2]:
train_merchant_data = pd.read_csv("train_merchant_data-1561627820784.csv", sep = ',', header = 0, na_values = ['unknown', '?'])
train_order_data = pd.read_csv("train_order_data-1561627847149.csv", sep = ',', header = 0, na_values = 'unknown')


train_data = pd.read_csv("train-1561627878332.csv", sep = ',', header = 0, na_values = 'unknown')
ip_boundaries_countries = pd.read_csv("ip_boundaries_countries-1561628631121.csv", sep = ',', header = 0, na_values = 'unknown')


In [3]:
print(train_data.shape)
print(train_order_data.shape)
print(train_data.shape)
print(ip_boundaries_countries.shape)

(54213, 2)
(54213, 7)
(54213, 2)
(138846, 3)


In [4]:
train_merge=pd.merge(train_merchant_data,train_order_data, how='outer',on='Merchant_ID')

In [5]:
merge_ip=pd.merge(train_data,train_merge, how='outer',on='Merchant_ID')

In [6]:
merge_ip.head()

,Merchant_ID,Fraudster,Ecommerce_Provider_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method
0,221592,0,1746213,2018-07-01 06:43:23,QDZKUGCGLVPZG,F,42,157.217.17.33,149173,6641fb40fd59,2018-10-29 17:58:25,205,Direct,Credit Card
1,316935,1,1746213,2018-01-18 01:10:42,GCCLHNBNIJJGW,M,42,46.8.62.247,117942,09ce4ab420a6,2018-03-24 20:15:57,88,Ads,Cash On Delivery
2,38454,1,1746213,2018-07-16 07:56:34,DAXDXWHPMSUJG,M,19,222.110.0.90,125604,41274e2a70c0,2018-09-19 15:13:54,100,Direct,Credit Card
3,214437,1,1746213,2018-04-28 17:08:09,XPBKQAAEUPZAO,F,35,104.242.92.50,155981,e8538b380577,2018-05-27 23:24:42,60,Direct,Debit Card
4,296240,1,1746213,2018-01-09 00:26:19,VGGTRBYDZMCPN,F,27,175.104.225.24,142401,72905bb38554,2018-01-11 00:26:20,88,Ads,E-wallet


In [7]:
merge_ip.shape

(54213, 14)

In [8]:
merge_ip.describe(include='all')

,Merchant_ID,Fraudster,Ecommerce_Provider_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method
count,54213.000000,54213.00000,54213.0,54213,54213,54213,54213.000000,54213,54213.000000,54213,54213,54213.000000,54213,54213
unique,NaN,NaN,NaN,54213,51291,2,NaN,52028,NaN,54213,54161,NaN,3,5
top,NaN,NaN,NaN,2018-02-23 11:38:16,VIPZYJGMVMXOF,M,NaN,115.241.135.193,NaN,d8820adf185f,2018-07-13 16:52:14,NaN,SEO,Credit Card
freq,NaN,NaN,NaN,1,9,31761,NaN,9,NaN,1,2,NaN,21884,21844
mean,200395.176212,0.09269,1746213.0,NaN,NaN,NaN,33.122240,NaN,137966.285208,NaN,NaN,92.230240,NaN,NaN
std,115398.486895,0.29000,0.0,NaN,NaN,NaN,8.630091,NaN,15563.516156,NaN,NaN,45.673263,NaN,NaN
min,2.000000,0.00000,1746213.0,NaN,NaN,NaN,18.000000,NaN,111234.000000,NaN,NaN,22.000000,NaN,NaN
25%,100997.000000,0.00000,1746213.0,NaN,NaN,NaN,27.000000,NaN,124471.000000,NaN,NaN,55.000000,NaN,NaN
50%,200574.000000,0.00000,1746213.0,NaN,NaN,NaN,32.000000,NaN,137864.000000,NaN,NaN,88.000000,NaN,NaN
75%,300407.000000,0.00000,1746213.0,NaN,NaN,NaN,39.000000,NaN,151405.000000,NaN,NaN,122.000000,NaN,NaN


In [9]:
merge_ip.isnull().sum()

Merchant_ID                   0
Fraudster                     0
Ecommerce_Provider_ID         0
Merchant_Registration_Date    0
Registered_Device_ID          0
Gender                        0
Age                           0
IP_Address                    0
Customer_ID                   0
Order_ID                      0
Date_of_Order                 0
Order_Value_USD               0
Order_Source                  0
Order_Payment_Method          0
dtype: int64

In [10]:
merge_ip.drop('Ecommerce_Provider_ID', axis=1, inplace=True)

In [11]:
def ip_to_int(ip_ser):
    ips = ip_ser.str.split('.', expand=True).astype(np.int16).values
    mults = np.tile(np.array([24, 16, 8, 0]), len(ip_ser)).reshape(ips.shape)
    return np.sum(np.left_shift(ips, mults), axis=1)

In [12]:
merge_ip.dtypes

Merchant_ID                    int64
Fraudster                      int64
Merchant_Registration_Date    object
Registered_Device_ID          object
Gender                        object
Age                            int64
IP_Address                    object
Customer_ID                    int64
Order_ID                      object
Date_of_Order                 object
Order_Value_USD                int64
Order_Source                  object
Order_Payment_Method          object
dtype: object

In [13]:
for col in ['Merchant_Registration_Date','Date_of_Order']:
    merge_ip[col]=merge_ip[col].astype('datetime64[ns]')

In [14]:
merge_ip.dtypes

Merchant_ID                            int64
Fraudster                              int64
Merchant_Registration_Date    datetime64[ns]
Registered_Device_ID                  object
Gender                                object
Age                                    int64
IP_Address                            object
Customer_ID                            int64
Order_ID                              object
Date_of_Order                 datetime64[ns]
Order_Value_USD                        int64
Order_Source                          object
Order_Payment_Method                  object
dtype: object

In [15]:
for col in ['Gender', 'Order_Source', 'Order_Payment_Method','Registered_Device_ID', 'IP_Address', 'Order_ID', 'Fraudster']:
    merge_ip[col]= merge_ip[col].astype('category')

In [16]:
merge_ip.dtypes

Merchant_ID                            int64
Fraudster                           category
Merchant_Registration_Date    datetime64[ns]
Registered_Device_ID                category
Gender                              category
Age                                    int64
IP_Address                          category
Customer_ID                            int64
Order_ID                            category
Date_of_Order                 datetime64[ns]
Order_Value_USD                        int64
Order_Source                        category
Order_Payment_Method                category
dtype: object

In [17]:
merge_ip.drop(['Merchant_Registration_Date','Date_of_Order'], axis=1, inplace=True)

In [18]:
merge_ip.dtypes

Merchant_ID                int64
Fraudster               category
Registered_Device_ID    category
Gender                  category
Age                        int64
IP_Address              category
Customer_ID                int64
Order_ID                category
Order_Value_USD            int64
Order_Source            category
Order_Payment_Method    category
dtype: object

In [19]:
cat_cols=list(merge_ip.select_dtypes('category').columns)
num_cols=list(merge_ip.columns.difference(cat_cols))

In [20]:
print(cat_cols)

['Fraudster', 'Registered_Device_ID', 'Gender', 'IP_Address', 'Order_ID', 'Order_Source', 'Order_Payment_Method']


In [21]:
print(num_cols)

['Age', 'Customer_ID', 'Merchant_ID', 'Order_Value_USD']


In [22]:
target_var= cat_cols.pop(0)

In [23]:
print(target_var)

Fraudster


In [31]:
# PipLine
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Categorical Pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'most_frequent', fill_value = 'missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

## or use Transform Merger instead of 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)])

In [32]:
clf_logreg = Pipeline(steps=[('preprocessor', preprocessor),
                             ('classifier', LogisticRegression())
                            ])


In [33]:
X, y = merge_ip.loc[:,merge_ip.columns!='Fraudster'], merge_ip.loc[:,'Fraudster']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123) 

In [34]:
clf_logreg.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [35]:
train_pred = clf_logreg.predict(X_train)
test_pred = clf_logreg.predict(X_test)

In [36]:
confusion_matrix= confusion_matrix(y_train, train_pred)

In [37]:
print(confusion_matrix)

[[34445     2]
 [ 2176  1326]]


In [41]:
#accuracy_score=(confusion_matrix[0,0]+ confusion_matrix[1,1])/(confusion_matrix[0,0]+confusion_matrix[0,1]+confusion_matrix[1,1]+confusion_matrix[1,0])

#TPR= confusion_matrix[0,0]/(confusion_matrix[0,0]+confusion_matrix[0,1])

In [42]:
test_merchant_data = pd.read_csv("test_merchant_data-1561627903902.csv", sep = ',', header = 0, na_values = ['unknown', '?'])
test_order_data = pd.read_csv("test_order_data-1561627931868.csv", sep = ',', header = 0, na_values = 'unknown')

test_data = pd.read_csv("test-1561627952093.csv", sep = ',', header = 0, na_values = 'unknown')

In [43]:
test_merge=pd.merge(test_merchant_data,test_order_data, how='outer',on='Merchant_ID')

merge_test=pd.merge(test_data,test_merge, how='outer',on='Merchant_ID')

In [44]:
merge_test.head()

,Merchant_ID,Ecommerce_Provider_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method
0,53637,1746213,2018-02-11 20:50:29,PTMLBENYMQCTV,F,40,134.162.124.62,157068,a032de091f51,2018-03-09 09:09:42,60,SEO,Credit Card
1,243517,1746213,2018-04-06 13:19:39,HMCLDZUZPWZRR,M,39,152.76.98.87,112534,0d563f0606d6,2018-06-13 20:11:19,28,SEO,Credit Card
2,343640,1746213,2018-01-12 16:47:49,VLGSMAPXISSEJ,M,23,31.202.3.255,148774,18fb0fa888b6,2018-04-14 04:19:36,75,Ads,Debit Card
3,69889,1746213,2018-02-19 21:58:52,ZINHISBBOKQXT,M,34,12.242.168.185,114528,ed1eb920d721,2018-04-01 13:55:44,98,SEO,Credit Card
4,125706,1746213,2018-05-17 15:50:19,WFKEAOTPHTYEO,M,20,26.61.210.47,120940,313dcf962627,2018-05-25 02:08:18,35,SEO,Credit Card


In [45]:
print(test_merge.shape)

(13554, 13)


In [46]:
print(merge_test.shape)

(13554, 13)


In [47]:
merge_ip=ip_to_int(merge_test.IP_Address)

In [48]:
for col in ['Merchant_Registration_Date','Date_of_Order']:
    merge_test[col] = merge_test[col].astype('datetime64[ns]')

In [49]:
for col in ['Merchant_Registration_Date','Date_of_Order']:
    merge_test[col]= merge_test[col].dt.date

In [50]:
merge_test.dtypes

Merchant_ID                    int64
Ecommerce_Provider_ID          int64
Merchant_Registration_Date    object
Registered_Device_ID          object
Gender                        object
Age                            int64
IP_Address                    object
Customer_ID                    int64
Order_ID                      object
Date_of_Order                 object
Order_Value_USD                int64
Order_Source                  object
Order_Payment_Method          object
dtype: object

In [51]:
for col in ['Gender', 'Order_Source', 'Order_Payment_Method','Registered_Device_ID', 'IP_Address', 'Order_ID', 'Merchant_Registration_Date','Date_of_Order']:
    merge_test[col] = merge_test[col].astype('category')

In [52]:
merge_test.drop(['Merchant_Registration_Date','Date_of_Order'], axis=1,inplace=True)

In [53]:
unseen = clf_logreg.predict(merge_test)

In [ ]:
unseen_df= pd.DataFrame(unseen, columns=['Frauster'])

In [ ]:
final_df= pd.concat([test_data, unseen_df])

In [ ]:
pd.DataFrame(unseen).to_csv('Fraud.csv')

In [ ]:
pd.DataFrame(final_df).to_csv('Fraud.csv')

In [ ]:
f1_Train = f1_score(y_true=y_train, y_pred = train_pred, average='weighted')

In [ ]:
f1_Train

In [57]:
f1_Test = f1_score(y_true=y_test, y_pred =test_pred, average='weighted')

In [58]:
f1_Test

0.8834984524543031